In [23]:
# made by self
from lib import gridmap as gm
from lib import makepath as mp
from lib import cartrobot as cb

import numpy as np
import math


def normalize(data):
    data = np.array(data)
    max_data = max(data)
    min_data = min(data)

    if max_data == min_data:
        data = [0.0 for i in range(len(data))]
    else:
        data = (data - min_data) / (max_data - min_data)

    return data

def correction_ang(angle):
    if angle > math.pi:
        while angle > math.pi:
            angle -= 2*math.pi
    elif angle < -math.pi:
        while angle < -math.pi:
            angle += 2*math.pi

    return angle

class Evaluator():
    def __init__(self):
        self.ang_scores = []
        self.obs_scores = []
        self.vel_scores = []
        self.scores     = []
        
    def calc_score(self, path, grid_map, goal):
        self.calc_obs_score(path, grid_map)
        self.calc_vel_score(path[0][3])
        self.calc_ang_score(path[-1], goal[0], goal[1])
        
    def calc_obs_score(self, path, grid_map):
        pass
    
    def calc_vel_score(self, v):
        self.vel_scores.append(v)
    
    def calc_ang_score(self, last_status, g_x, g_y):
        self.ang_scores.append(
            math.pi-abs(
                correction_ang(
                    math.atan2(g_y-last_status[1], g_x-last_status[1]) - last_status[2]
                )
            )
        )

if __name__ == "__main__":
    import csv
    start = time.time()
    cartbot = cb.main(0, 0, math.pi/2)
    #LRFからのデータに変える
    obstacles = [list((np.random.rand(2)-0.5)*10) for i in range(50)]
    goal = [10, 10]
    
    for _ in range(1):
        evals = Evaluator()
        paths = mp.main(cartbot.status)
        grid_map = gm.main(paths, obstacles)

        for path in paths:
            evals.calc_score(path, grid_map, goal)
    finished = time.time()-start
    print(finished)
    
    with open("gridmap.csv", "w") as file:
        write = csv.writer(file, lineterminator="\n")
        write.writerows(grid_map)

0.11123895645141602


[[-18.847742418634372, -10.799811945067372],
 [-6.784203274842154, 9.105153277803064],
 [-14.945796544250701, -11.51097753250832],
 [-14.005387018873822, 12.702937711227385],
 [24.28246341930192, -17.592246462144256],
 [-5.752280768503598, 5.712798968679006],
 [2.68750758903738, -12.7801317531898],
 [20.792519259677743, 12.69915689937759],
 [23.48885027269208, 18.332983146962217],
 [-3.1908109473414825, -0.7055922658647518],
 [2.4503575072829467, 23.821814015046446],
 [-19.889562226035125, 14.811842309839946],
 [13.322522401226106, 15.053730148654237],
 [-23.515011045743115, 14.924437412695003],
 [17.587219970141454, -6.576890195853736],
 [12.847859170693393, 6.276619038278425],
 [5.739032511020187, 10.306822639382156],
 [-9.76698025497244, 21.494234033498437],
 [-11.579792568204622, 21.212123155949502],
 [0.5296449544314352, -8.480382936588738],
 [-22.135602575233758, -16.68875032674486],
 [9.35369172849142, -21.558054961059515],
 [19.6065084748419, 23.937901823162978],
 [23.065007433

In [34]:
import numpy as np
b = np.array(a)
b[0:5,0:5]

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

In [33]:
b

array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])